https://www.analyticsvidhya.com/blog/2020/07/transfer-learning-for-nlp-fine-tuning-bert-for-text-classification/

https://curiousily.com/posts/sentiment-analysis-with-bert-and-hugging-face-using-pytorch-and-python/

# Getting the data

https://scikit-learn.org/stable/modules/model_evaluation.html

In [1]:
!git clone https://github.com/YasaminAbbaszadegan/NLP_Forum_Recomendation_Engine.git

Cloning into 'NLP_Forum_Recomendation_Engine'...
remote: Enumerating objects: 73, done.
remote: Total 73 (delta 0), reused 0 (delta 0), pack-reused 73
Unpacking objects: 100% (73/73), done.


In [2]:
%cd /content/NLP_Forum_Recomendation_Engine/Dataset

/content/NLP_Forum_Recomendation_Engine/Dataset


In [3]:
ls

tudiabetes_20.csv


In [4]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from wordcloud import WordCloud
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:

df=pd.read_csv('tudiabetes_20.csv',index_col=0) #MyDF_final_main_comment.csv

In [6]:
#df=df.drop(['tags'],axis=1)
df

,Topic Title,Category,Tags,Leading Post,Post Replies,Created_at,Likes,Views,Replies
0,About the Social Clubs category,Social Clubs,[],Social Clubs are places for people with simila...,[],"Jul 26, 2018 9:24 pm",0,722,2
1,Latest Movie You’ve Seen?,Social Clubs,['Movies and Entertainment Media'],"Hi Everyone,\nWhat is the latest movie that yo...",['My wife and I see about one movie a week. L...,"Mar 12, 2019 12:45 am",0,578,5
2,Anyone doing the Broad St. Run in philly,Social Clubs,['Running'],got my notification yesterday. time to get bac...,[],0,0,0,0
3,Embroidery…. just starting out,Social Clubs,['Carefully Crafted'],Love love love creating!\nAfter years of not s...,"['That’s beautiful!', 'Thanks, so loving a dar...","Feb 4, 2019 11:31 am",4,505,2
4,Home theatre,Social Clubs,['Movies and Entertainment Media'],If anyone is approaching retirement and wonder...,[],0,0,0,0
...,...,...,...,...,...,...,...,...,...
3694,Frequent Occlusions with Tandem X2,Diabetes Technology,[],A few months ago my son switched from a Medtro...,"['What infusion sets are he using?', 'The Tand...","Dec 11, 2019 3:52 pm",7,700,15
3695,Any loopers using this button?,Diabetes Technology,['DIY Closed Loop Systems'],Anybody using this button (right of the meal b...,"['I use this button many times, every day. It’...","Dec 7, 2019 11:46 am",0,645,24
3696,Which infusion set?,Diabetes Technology,['Insulin Pumps'],I have Tandem T2 pump. Started using the steel...,['I use the Autosoft 30. A spring sends it in...,"Dec 12, 2018 6:10 pm",7,1.8k,17
3697,Availability of diabetes products in Canada. I...,Diabetes Technology,[],So I love new advancements in D-technology. Bu...,['Yes - frustrated.\nI would like to try Tresi...,"Oct 6, 2016 9:14 pm",16,7.0k,33


# Remove '[ ]' in Tags and Post Replies

In [7]:
df['Tags'] = df['Tags'].str.replace("[", "")
df['Tags'] = df['Tags'].str.replace("]", "")
df['Tags'] = df['Tags'].str.replace("'", "")
df['Post Replies'] = df['Post Replies'].str.replace("[", "")
df['Post Replies'] = df['Post Replies'].str.replace("]", "")
df['Post Replies'] = df['Post Replies'].str.replace("'", "")


In [8]:
df_cat = df[df.Category != 'Social Clubs']
df_cat = df_cat[df.Category != 'TuDiabetes Website']
df_cat = df_cat[df.Category != 'Welcome']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  This is separate from the ipykernel package so we can avoid doing imports until


In [9]:
my_categories = [ 'Type 1 and LADA',
'Type 2', 'Diabetes and Pregnancy', 'Community', 'Food',
'Treatment', 'Diabetes Technology']

df_cat['comment_topic'] = df_cat['Topic Title'] + ' ' + df_cat['Tags'] + ' ' + df_cat['Leading Post'] + ' ' + df_cat['Post Replies']
df_cat['comment_tag'] = df_cat['Tags'] + ' ' + df_cat['Leading Post'] + ' ' + df_cat['Post Replies']
df_cat['comment_lead'] = df_cat['Leading Post'] + ' ' + df_cat['Post Replies']


In [10]:
df_cat['comment_topic'][0:1].str.strip().str.replace('\n', ' ').values

array(['Type 1 Diabetes Resources  Information on this page represents a sample of the content available online for people with type 1 diabetes and LADA, curated by members of the community.. This is not an exhaustive list of the resources offered online, but may be viewed as a good place to begin your search. General Information A good place to start getting familiar with type 1 diabetes. What is Type 1 Diabetes? 151 What is Diabetes? The Difference Between Type 1 and Type 2 102 US Centers for Disease Control diabetes resources 35 Positive Autoantibody Tests Indicate Type 1 Autoimmune Diabetes 64 How to properly and safely dispose of needles 51    Other Resources   Type 1 Diabetes Education 81 (Integrated Diabetes Services)  diaTribe 52 - research and product news for PWD Children with Diabetes 82  Strategies for Insulin Injection Therapy in Diabetes Self-Management (AADE) ACT1: Adults Coping with Type 1 Diabetes 141  Type 1 Diabetes section 50 (Islets of Hope)  Taking Control of Your

#Clean

In [11]:
!pip install contractions

     |████████████████████████████████| 245kB 28.8MB/s 
     |████████████████████████████████| 327kB 52.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp36-cp36m-linux_x86_64.whl size=84338 sha256=d9d12bd7d2bb0ce6e88a3e640a1cd1bae5ad47009ca04e85fb9657eab0644751
  Stored in directory: /root/.cache/pip/wheels/e4/ab/f7/cb39270df8f6126f3dd4c33d302357167086db460968cfc80c
Successfully built pyahocorasick


In [12]:


import contractions

import string
def clean_text(text):
    #Make text lowercase, remove punctuation and remove words containing numbers.
    text=re.sub("[\(\[].*?[\)\]]", "",text)
    text=re.sub(r'#\w+ ?', '', text ) ## remove hashtag words
    text = re.sub('<.*?>','', text)
    text = text.lower()
    # text = re.sub('[%s]' % re.escape(string.punctuation), '',text)#remove punctuation
   # Get rid of some additional punctuation.
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub(' — ', '', text)
    return (text)

clean = lambda x: clean_text(x)

def full_clean(s):
  df = s.apply(lambda x: [contractions.fix(word) for word in str(x).split()])
  df= df.map(lambda x: ' '.join(x))
  stop_words.update(['hi','hello']) 

  df=df.apply(clean)
  df = df.apply(lambda x: ' '.join([word for word in str(x).split() if word not in (stop_words)]))
  
  return(pd.DataFrame(df))


df_cat['comment_topic_C'] =full_clean(df_cat['comment_topic'])
df_cat['comment_tag_C'] = full_clean(df_cat['comment_tag'])
df_cat['comment_lead_C']= full_clean(df_cat['comment_lead'])


In [13]:
df_cat.head()

,Topic Title,Category,Tags,Leading Post,Post Replies,Created_at,Likes,Views,Replies,comment_topic,comment_tag,comment_lead,comment_topic_C,comment_tag_C,comment_lead_C
192,Type 1 Diabetes Resources,Type 1 and LADA,,Information on this page represents a sample o...,,"Dec 21, 2009 3:04 pm",2,9.2k,6,Type 1 Diabetes Resources Information on this...,Information on this page represents a sample ...,Information on this page represents a sample o...,type 1 diabetes resources information page rep...,information page represents sample content ava...,information page represents sample content ava...
193,About the Type 1 and LADA category,Type 1 and LADA,,The Type 1 Diabetes and LADA category is inten...,,"Apr 14, 2015 5:51 pm",1,7.1k,1,About the Type 1 and LADA category The Type 1...,The Type 1 Diabetes and LADA category is inte...,The Type 1 Diabetes and LADA category is inten...,type 1 lada category type 1 diabetes lada cate...,type 1 diabetes lada category intended discuss...,type 1 diabetes lada category intended discuss...
194,Are you concerned about Covid-19 Vaccine might...,Type 1 and LADA,,I am wondering if others with multiple autoimm...,"Yeah. However, given that this is a pandemic, ...","Jan 14, 2021 6:40 am",32,349,17,Are you concerned about Covid-19 Vaccine might...,I am wondering if others with multiple autoim...,I am wondering if others with multiple autoimm...,concerned covid-19 vaccine might wake autoimmu...,wondering others multiple autoimmune diseases ...,wondering others multiple autoimmune diseases ...
195,Covid Vaccine - are you getting it?,Type 1 and LADA,,Hello!\nI wanted to get people who have type 1...,I am Type 2 but the question is just a relevan...,"Jan 2, 2021 8:39 pm",203,1.9k,120,Covid Vaccine - are you getting it? Hello!\nI...,Hello!\nI wanted to get people who have type ...,Hello!\nI wanted to get people who have type 1...,covid vaccine - getting it? hello! wanted get ...,hello! wanted get people type 1 opinions wheth...,hello! wanted get people type 1 opinions wheth...
196,Bolusing for pasta - big fail,Type 1 and LADA,,How do you guys bolus for pasta? I had some e...,"With pasta (any kind of spaghetti, light on th...","Jan 16, 2021 6:30 am",11,147,13,Bolusing for pasta - big fail How do you guys...,How do you guys bolus for pasta? I had some ...,How do you guys bolus for pasta? I had some e...,bolusing pasta - big fail guys bolus pasta? el...,"guys bolus pasta? elbow pasta tonight, little ...","guys bolus pasta? elbow pasta tonight, little ..."


In [14]:
my_categories = [ 'Type 1 and LADA',
'Type 2', 'Diabetes and Pregnancy', 'Community', 'Food',
'Treatment', 'Diabetes Technology']

In [15]:
df_cat['Category']=df_cat['Category'] .apply(my_categories.index)

# Using Colab GPU for Training

Google Colab offers free GPUs and TPUs! Since we'll be training a large neural network it's best to take advantage of this (in this case we'll attach a GPU), otherwise training will take a very long time.

A GPU can be added by going to the menu and selecting:

`Edit 🡒 Notebook Settings 🡒 Hardware accelerator 🡒 (GPU)`

Then run the following cell to confirm that the GPU is detected.

In [16]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In order for torch to use the GPU, we need to identify and specify the GPU as the device. Later, in our training loop, we will load data onto the device.

In [17]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# Installing the Hugging Face Library

Next, let's install the transformers package from Hugging Face which will give us a pytorch interface for working with BERT. (This library contains interfaces for other pretrained language models like OpenAI's GPT and GPT-2.) We've selected the pytorch interface because it strikes a nice balance between the high-level APIs (which are easy to use but don't provide insight into how things work) and tensorflow code (which contains lots of details but often sidetracks us into lessons about tensorflow, when the purpose here is BERT!).

At the moment, the Hugging Face library seems to be the most widely accepted and powerful pytorch interface for working with BERT. In addition to supporting a variety of different pre-trained transformer models, the library also includes pre-built modifications of these models suited to your specific task. For example, in this tutorial we will use BertForSequenceClassification.

The library also includes task-specific classes for token classification, question answering, next sentence prediciton, etc. Using these pre-built classes simplifies the process of modifying BERT for your purposes.

In [18]:
!pip install transformers

     |████████████████████████████████| 1.8MB 15.7MB/s 
     |████████████████████████████████| 890kB 54.7MB/s 
     |████████████████████████████████| 2.9MB 53.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=098ff329ca75f0a0391c85972ae635697b49e2dc4ffeb532e710ef2309fd083b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


# BERT Tokenizer

To feed our text to BERT, it must be split into tokens, and then these tokens must be mapped to their index in the tokenizer vocabulary.

The tokenization must be performed by the tokenizer included with BERT--the below cell will download this for us. We'll be using the "uncased" version here

In [19]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [20]:
df_test=df_cat.loc[df_cat.Category == 0].sample(5)[['comment_topic_C', 'Category']]
df_test

,comment_topic_C,Category
415,long take long take find diabetic groove. long...,0
198,urinating lot night anyone else issue urinatin...,0
599,frio options keeping insulin cool using frio c...,0
646,"starting insulin? lada go insulin right away, ...",0
629,type 1 dm survey/input needed touro university...,0


In [21]:
# Get the lists of sentences and their labels.
sentences = df_test.comment_topic_C.values
labels = df_test.Category.values

In [22]:
sentences[0]

'long take long take find diabetic groove. long felt confortable confident within many aspects diabetes felt like knew able speak, ans wer preform tasks infront people diabetes. couple decades, guess really totally comfortable confident advent cgm really know going 24/7/365, 27 years, finally able catch modern technology. without insurance coverage, old school mostly trying survive insulin., +18 yearsthough asked first 10 years might answered differently. know better age/wisdom combo thereof finally settled , dx age 11. age 25, came skin., realized today 30 year t1d anniversary. huh.\\ni embarrassed/shy t1d child/teenager/young adult. tried inconspicuous possible talk unless absolutely to. nowwhatever. try hide it. anyone asks, talk it.\\nas confident, wellit part life deal best somewhat successful part. still days feel like idea doing., congrats hitting 30 year survival mark.\\n\\n\\n\\n brianna1:\\n\\ni embarrassed/shy t1d child/teenager/young adult. tried inconspicuous possible talk

In [ ]:
# Print the original sentence.
print(' Original: ', sentences[0])

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(sentences[0]))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

 Original:  dawn effect reading posts people posting basal rates hoping get tslim soon let pump alter basal rates old mm pump strong dawn effect think realized much basal midnight 075u 330 125u 700 10 noon 125u 500 pm 10 dawn effect requires double insulin rate way eating anything see many others changing rates much sometimes even wake increased sugar increased insulin often wakes around 4 much choice though either let sugar go 200 wake surge energy sort used drink night like twoshotsof whiskey tend miss dawn effect completely hardly plausible solution daily basis drink need remember alter basal rate avoid lows really hoping new pump solve crap cgm really helps see trends know anything really tslim omnipod dp worse used 3 times normal insulin couple hours combat literally pump giving 24 units couple hours row starting 5 amni still mild dp aggravating sometimes likes surge weeks goes away actually aggravating sometimes least consistently high 7 settings right one possibly going away 6 7

When we actually convert all of our sentences, we'll use the tokenize.encode function to handle both steps, rather than calling tokenize and convert_tokens_to_ids separately.

Before we can do that, though, we need to talk about some of BERT's formatting requirements.

# Required Formatting